In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
! rm ./test_db3.db

In [3]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v['description'], interval_number=v['interval_number'], 
                                      top=v['top'], base=v['base']))
   # components_values = association_proxy(
   #     'components', 'value',
   #     creator=lambda k, v: Component(id=k, description=v['description']))
    
class Position(Base):
    """The Position table"""
    __tablename__ = 'Positions'
    id = Column(Integer, primary_key=True)
    # interval = relationship('Interval') # , uselist=False, back_populates='top')
    upper = Column(Float(32))
    middle = Column(Float(32))
    lower = Column(Float(32))
    
class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    id = Column(Integer, primary_key=True)
    borehole = Column(String(32), ForeignKey('Boreholes.id'))
    # Note that this could be a numeric ID as well
    interval_number = Column(Integer)
    components = relationship('Component',secondary='Linkintervalcomponent')
    description = Column(String(32))
    top_id = Column(Integer, ForeignKey('Positions.id'))
    top = relationship(Position, foreign_keys=[top_id])
    base_id = Column(Integer, ForeignKey('Positions.id'))
    base = relationship(Position, foreign_keys=[base_id])

class Component(Base):
    """The Component table"""
    __tablename__ = 'Components'
    id = Column(String(32), primary_key=True)
    intervals = relationship(Interval,secondary='Linkintervalcomponent')
    description = Column(String(32))

class LinkIntervalComponent(Base):
    __tablename__ = 'Linkintervalcomponent'

    int_id = Column(
        Integer, 
        ForeignKey('Intervals.id'), 
        primary_key = True)

    comp_id = Column(
       Integer, 
       ForeignKey('Components.id'), 
       primary_key = True)

In [4]:
engine = create_engine('sqlite:///test_db3.db', echo=True)

In [5]:
Base.metadata.create_all(engine)

2020-12-18 14:26:22,022 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-18 14:26:22,023 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 14:26:22,026 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-18 14:26:22,027 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 14:26:22,029 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-18 14:26:22,030 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 14:26:22,031 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-18 14:26:22,032 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 14:26:22,033 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2020-12-18 14:26:22,034 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 14:26:22,036 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2020-12-18 14:26:22,037 INFO sqlalchemy.engine.base.Engine ()
2020-12-18

In [6]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [7]:
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
from striplog import Lexicon, Striplog

In [9]:
lexicon = Lexicon.default()

In [10]:
borehole_dict = {'F01':'test.las', 'F02':'test.las'} # boreholes to insert into the db

In [11]:
int_id = 0
bh_id = 0
pos_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
pos_dict = {}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(Borehole(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        top = Position(id=pos_id, upper=interval.top.upper, middle=interval.top.middle,  lower=interval.top.lower)
        base = Position(id=pos_id+1, upper=interval.base.upper, middle=interval.base.middle,  lower=interval.base.lower)
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number, 
                          'top': top, 'base': base
                         }})
        interval_number+=1
        int_id += 1
        pos_id += 2

    print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

{0: {'description': 'Anhydrite', 'interval_number': 0, 'top': <__main__.Position object at 0x7fb6cd3534a8>, 'base': <__main__.Position object at 0x7fb6cd353240>}, 1: {'description': 'Sandstone, grey, vf-f', 'interval_number': 1, 'top': <__main__.Position object at 0x7fb6cd353278>, 'base': <__main__.Position object at 0x7fb6cd353d30>}, 2: {'description': 'Anhydrite', 'interval_number': 2, 'top': <__main__.Position object at 0x7fb6cd353ac8>, 'base': <__main__.Position object at 0x7fb6cd353630>}, 3: {'description': 'Dolomite', 'interval_number': 3, 'top': <__main__.Position object at 0x7fb6cd353780>, 'base': <__main__.Position object at 0x7fb6cd353978>}, 4: {'description': 'Anhydrite', 'interval_number': 4, 'top': <__main__.Position object at 0x7fb6cd3539e8>, 'base': <__main__.Position object at 0x7fb6cd353a90>}, 5: {'description': 'Sandstone, grey, vf-f', 'interval_number': 5, 'top': <__main__.Position object at 0x7fb6cd353be0>, 'base': <__main__.Position object at 0x7fb6cd353a20>}, 6: {

In [12]:
strip[0].base

upper,51.24117647058824
middle,51.24117647058824
lower,51.24117647058824


In [13]:
for id in components.keys():
    print(id, components[id])

0 {'lithology': 'siltstone', 'colour': 'grey'}
1 {'lithology': 'anhydrite'}
2 {'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}
3 {'lithology': 'dolomite'}
4 {'lithology': 'siltstone', 'colour': 'red'}
5 {'lithology': 'limestone'}


In [14]:
#with session_scope() as session:
#    for pos in pos_dict.values():
        
#        session.add(pos)

with session_scope() as session:
    for bh in boreholes:
        session.add(bh)
        #for iv in bh.intervals:
        #    print(bh.intervals[iv].components[0])
        
with session_scope() as session:
    for id in components.keys():
        new_compo = Component(id=id, description=components[id].summary()) 
        session.add(new_compo)

2020-12-18 14:26:23,306 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 14:26:23,319 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower) VALUES (?, ?, ?, ?)
2020-12-18 14:26:23,320 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295)  ... displaying 10 of 100 total bound parameter sets ...  (98, 131.47647058823532, 131.47647058823532, 131.47647058823532), (99, 135.9, 135.9, 135.9))
2020-12-18 14:26:23,326 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-18 14:26:23,327 INFO